<a href="https://colab.research.google.com/github/MrBigBrane/Machine-Learning/blob/main/AI_vs_Real_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# import libraries
import numpy as np
import pandas as pd

import os
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset

# for getting summary info on models
from torchsummary import summary

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

import kagglehub

In [ ]:
# use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Data Preprocessing

In [3]:
# from google.colab import files
# files.upload()

# ! rm -r ~/.kaggle
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

# !kaggle competitions download -c detect-ai-vs-human-generated-images

# !unzip /content/detect-ai-vs-human-generated-images -d /content/aivshuman

Saving kaggle.json to kaggle (1).json
detect-ai-vs-human-generated-images.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/detect-ai-vs-human-generated-images.zip
  inflating: /content/aivshuman/test.csv  
  inflating: /content/aivshuman/train.csv  


In [23]:
# Download latest version
path = kagglehub.dataset_download("alessandrasala79/ai-vs-human-generated-dataset")

print("Path to dataset files:", path)

100%|██████████| 9.76G/9.76G [02:04<00:00, 83.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alessandrasala79/ai-vs-human-generated-dataset/versions/4


In [34]:
test = pd.read_csv('/root/.cache/kagglehub/datasets/alessandrasala79/ai-vs-human-generated-dataset/versions/4/test.csv')
train = pd.read_csv('/root/.cache/kagglehub/datasets/alessandrasala79/ai-vs-human-generated-dataset/versions/4/train.csv')

train = train[['file_name', 'label']]
train.columns = ['id', 'label']

ids = test.index.tolist()
ids = [i+1 for i in ids]

In [31]:
img_name = os.path.join('/root/.cache/kagglehub/datasets/alessandrasala79/ai-vs-human-generated-dataset/versions/4', train.iloc[300, 0])
image = Image.open(img_name).convert('RGB')

In [35]:
class AIImageDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.dataframe.iloc[idx, 1]
        return image, label

In [54]:
train_transforms = transforms.Compose([
    transforms.Resize(28),  # Resize to match ConvNeXt preprocessing
    transforms.RandomResizedCrop(28),
    # transforms.RandomHorizontalFlip(),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = AIImageDataset(train, '/root/.cache/kagglehub/datasets/alessandrasala79/ai-vs-human-generated-dataset/versions/4', transform=train_transforms)

In [55]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [56]:
# create a class for the model
def makeTheNet(printtoggle=False):

  class mnistnet(nn.Module):
    def __init__(self,printtoggle):
      super().__init__()

      # print toggle
      self.print = printtoggle

      ### -------------- feature map layers -------------- ###
      # first convolution layer
      self.conv1  = nn.Conv2d(3,32,3,padding=2)
      self.bnorm1 = nn.BatchNorm2d(32) # input the number of channels in this layer
      # output size: (28+2*2-3)/1 + 1 = 30/2 = 15 (/2 b/c maxpool)

      # second convolution layer
      self.conv2  = nn.Conv2d(32,64,3,padding=2)
      self.bnorm2 = nn.BatchNorm2d(64) # input the number of channels in this layer
      # output size: (15+2*2-3)/1 + 1 = 17/2 = 8 (/2 b/c maxpool)

      # third convolution layer
      self.conv3  = nn.Conv2d(64,128,3,padding=2)
      self.bnorm3 = nn.BatchNorm2d(128) # input the number of channels in this layer
      # output size: (8+2*2-3)/1 + 1 = 10/2 = 5 (/2 b/c maxpool)

      # fourth convolution layer
      self.conv4  = nn.Conv2d(128,256,3,padding=2)
      self.bnorm4 = nn.BatchNorm2d(256) # input the number of channels in this layer
      # output size: (4+2*2-3)/1 + 1 = 6/2 = 3 (/2 b/c maxpool)

      # fifth convolution layer
      self.conv5  = nn.Conv2d(256,512,3,padding=2)
      self.bnorm5 = nn.BatchNorm2d(512) # input the number of channels in this layer
      # output size: (3+2*2-3)/1 + 1 = 5/2 = 2 (/2 b/c maxpool)

      ### -------------- linear decision layers -------------- ###
      self.fc1 = nn.Linear(2*2*512,512)
      self.fc2 = nn.Linear(512, 10)

    def forward(self,x):

      if self.print: print(f'Input: {list(x.shape)}')

      # first block: convolution -> maxpool -> batchnorm -> relu
      x = F.max_pool2d(self.conv1(x),2)
      x = F.leaky_relu(self.bnorm1(x))
      if self.print: print(f'First CPR block: {list(x.shape)}')

      # second block: convolution -> maxpool -> batchnorm -> relu
      x = F.max_pool2d(self.conv2(x),2)
      x = F.leaky_relu(self.bnorm2(x))
      if self.print: print(f'Second CPR block: {list(x.shape)}')

      # third block: convolution -> maxpool -> batchnorm -> relu
      x = F.max_pool2d(self.conv3(x),2)
      x = F.leaky_relu(self.bnorm3(x))
      if self.print: print(f'Third CPR block: {list(x.shape)}')

      # fourth block: convolution -> maxpool -> batchnorm -> relu
      x = F.max_pool2d(self.conv4(x),2)
      x = F.leaky_relu(self.bnorm4(x))
      if self.print: print(f'Fourth CPR block: {list(x.shape)}')

      # fifth block: convolution -> maxpool -> batchnorm -> relu
      x = F.max_pool2d(self.conv5(x),2)
      x = F.leaky_relu(self.bnorm5(x))
      if self.print: print(f'Fourth CPR block: {list(x.shape)}')

      # reshape for linear layer
      nUnits = x.shape.numel()/x.shape[0]
      x = x.view(-1,int(nUnits))
      if self.print: print(f'Vectorized: {list(x.shape)}')

      # linear layers
      x = F.leaky_relu(self.fc1(x))
      x = self.fc2(x)
      if self.print: print(f'Final output: {list(x.shape)}')

      return x

  # create the model instance
  net = mnistnet(printtoggle)

  # loss function
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.Adam(net.parameters(),lr=.00001,weight_decay=5e-3)

  return net,lossfun,optimizer

In [57]:
X, y = next(iter(train_loader))
net,lossfun,optimizer = makeTheNet(printtoggle=True)
yTest = net(X)

Input: [32, 3, 28, 28]
First CPR block: [32, 32, 15, 15]
Second CPR block: [32, 64, 8, 8]
Third CPR block: [32, 128, 5, 5]
Fourth CPR block: [32, 256, 3, 3]
Fourth CPR block: [32, 512, 2, 2]
Vectorized: [32, 2048]
Final output: [32, 10]


In [ ]:
# a function that trains the model

def function2trainTheModel():

  # number of epochs
  numepochs = 1

  # create a new model
  net,lossfun,optimizer = makeTheNet()

  # send the model to the GPU
  net.to(device)

  # initialize losses
  trainLoss = torch.zeros(numepochs)
  testLoss  = torch.zeros(numepochs)
  trainErr  = torch.zeros(numepochs)
  testErr   = torch.zeros(numepochs)


  # loop over epochs
  for epochi in range(numepochs):

    # loop over training data batches
    net.train()
    batchLoss = []
    batchErr  = []
    for X,y in train_loader:

      # push data to GPU
      X = X.to(device)
      y = y.to(device)

      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss and error from this batch
      batchLoss.append(loss.item())
      batchErr.append( torch.mean((torch.argmax(yHat,axis=1) != y).float()).item() )
    # end of batch loop...

    # and get average losses and error rates across the batches
    trainLoss[epochi] = np.mean(batchLoss)
    trainErr[epochi]  = 100*np.mean(batchErr)



    # ### test performance
    # net.eval()
    # X,y = next(iter(validation_loader)) # extract X,y from test dataloader

    # # push data to GPU
    # X = X.to(device)
    # y = y.to(device)

    # with torch.no_grad(): # deactivates autograd
    #   yHat = net(X)
    #   loss = lossfun(yHat,y)

    # # get loss and error rate from the test batch
    # testLoss[epochi] = loss.item()
    # testErr[epochi]  = 100*torch.mean((torch.argmax(yHat,axis=1) != y).float()).item()

  # end epochs

  # function output
  return trainLoss,testLoss,trainErr,testErr,net

In [ ]:
trainLoss,testLoss,trainErr,testErr,net = function2trainTheModel()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,5))

ax[0].plot(trainLoss,'s-',label='Train')
ax[0].plot(testLoss,'o-',label='Test')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss (MSE)')
ax[0].set_title('Model loss')

ax[1].plot(trainErr,'s-',label='Train')
ax[1].plot(testErr,'o-',label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Error rates (%)')
ax[1].set_title(f'Final model test error rate: {testErr[-1]:.2f}%')
ax[1].legend()

plt.show()